Initializations:

In [1]:
%reload_ext snakeviz
%matplotlib inline
%run SBRG2.py

#Core
##Pauli Algebra
Pauli operator class. A Pauli operator is denoted as
$$\sigma=\mathrm{i}^{x\cdot z}\prod_{i\in Xs}X_i\prod_{i\in Zs}Z_i=\mathrm{i}^{x\cdot z}\prod_{i}X_i^{x_i}Z_i^{z_i},$$
where $x_i=\delta(i\in Xs)$, $z_i=\delta(i\in Zs)$ are vectors of 0,1, and $X_i$, $Z_i$ are the Pauli operator on site $i$.

Pauli operators can be created
- from Xs, Zs lists: `Pauli(Xs, Zs)`,
- or from dict of indices: `Pauli({i:mu, ...})`,
- or from list of indices: `Pauli([m0, mu1, ...])`.

In [2]:
[Pauli([0,1],[1,2]), Pauli({0:1,1:2,2:3}), Pauli([1,2,3])]

[<Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>]

The commutativity of two Pauli operators can be checked by `is_commute`

In [3]:
is_commute(Pauli([0,1,0]),Pauli([1,2,3]))

False

Merge two Pauli operators by indices. Coefficients are not calculated here. The coefficient can be restored from the commutation relations later.

In [4]:
pdot(Pauli([0,1,0]),Pauli([1,2,3]))

<Xs:[0] Zs:[1, 2]>

### Index Processing Unit (IPU)
Pauli indices are prossed in Fortran object IPU. Set two index arrays A and B by `ipu.setab(A, B)`.

In [2]:
A = Pauli([3,2,1,2,1])
print(A)
ipu.setab(array(A.Xs), array(A.Zs))

<Xs:[1, 2, 3, 4] Zs:[0, 1, 3]>


In [3]:
print(ipu.a[:ipu.na])
print(ipu.b[:ipu.nb])

[1 2 3 4]
[0 1 3]


Count the number of overlap indices by `dups = ipu.overlap()`.

In [4]:
ipu.overlap()

2

List operations
- Merge indices by the method `ipu.merge()`, 
- Combine indices by the method `ipu.combine()`,
- Intersect indices by the method `ipu.intersect()`.

The result is stored in `ipu.c[:ipu.nc]`.

In [3]:
ipu.merge()
print(ipu.c[:ipu.nc])
ipu.combine()
print(ipu.c[:ipu.nc])
ipu.intersect()
print(ipu.c[:ipu.nc])

[0 2 4]
[0 1 2 3 4]
[1 3]


###Pauli Monomial
A term of Pauli operator `Term.mat` with a constant multiple `Term.val`.

In [2]:
[Term(), Term(Pauli([1,2])),Term(Pauli([1,2]),-3.2)]

[1.0 <Xs:[] Zs:[]>, 1.0 <Xs:[0, 1] Zs:[1]>, -3.2 <Xs:[0, 1] Zs:[1]>]

Two Pauli monomials are compared by and only by their Pauli operators:

In [7]:
print(Term(Pauli([1,2]),2.) == Term(Pauli([1,2]),1.))
print(Term(Pauli([1,1]),2.) > Term(Pauli([1,2]),1.))

True
True


Consider $\sigma_A=\mathrm{i}^{x_A\cdot z_A} X_A Z_A$, $\sigma_B=\mathrm{i}^{x_B\cdot z_B} X_B Z_B$, then the product reads:
$$\sigma_A\sigma_B=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B+2 z_A\cdot x_B}\quad (X_A X_B) (Z_A Z_B)=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B}\quad \mathrm{i}^{x_C\cdot z_C}X_C Z_C=\mathrm{i}^{n^{C}_{AB}}\; \sigma_C,$$
where $X_C=X_AX_B$, $Z_C=Z_AZ_B$ and $z_C = (z_A+z_B)\mod 2$. The power of $\mathrm{i}$ is
$$n^{C}_{AB}=x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B$$

In [5]:
dot(Term(Pauli([1,2])),Term(Pauli([3,3])))

1.0 <Xs:[0, 1] Zs:[0]>

In [71]:
idot(Term(Pauli([2,0])),Term(Pauli([3,3])))

-1.0 <Xs:[0] Zs:[1]>

##Pauli Polynomial
Pauli polynomial `Poly` is a sum of Pauli monomial terms, with additional data keeping the order of Pauli operators and values. It can be created from a list of terms.

In [58]:
H = Poly([Term(Pauli([1,3,0]),1.0),
      Term(Pauli([0,1,1]),0.5),
      Term(Pauli([3,0,0]),0.2),
      Term(Pauli([0,3,3]),0.8)])

In [51]:
H.terms

[0.8 <Xs:[] Zs:[1, 2]>,
 1.0 <Xs:[0] Zs:[1]>,
 0.2 <Xs:[] Zs:[0]>,
 0.5 <Xs:[1, 2] Zs:[]>]

In [52]:
H.descending()

[1.0 <Xs:[0] Zs:[1]>,
 0.8 <Xs:[] Zs:[1, 2]>,
 0.5 <Xs:[1, 2] Zs:[]>,
 0.2 <Xs:[] Zs:[0]>]

In [16]:
H.imap

{0: {1.0 <Xs:[0] Zs:[1]>, 0.2 <Xs:[] Zs:[0]>},
 1: {0.8 <Xs:[] Zs:[1, 2]>, 1.0 <Xs:[0] Zs:[1]>, 0.5 <Xs:[1, 2] Zs:[]>},
 2: {0.8 <Xs:[] Zs:[1, 2]>, 0.5 <Xs:[1, 2] Zs:[]>}}

In [59]:
Rs = [Term(Pauli([0],[0,1]),-1.)]
print(Rs)

[-1.0 <Xs:[0] Zs:[0, 1]>]


In [60]:
H.forward(Rs)

In [6]:
H.extend([Term(Pauli([1],[0]),-0.1)])

In [61]:
H.remove(H.UVscale)

In [21]:
H.imap

{0: {1.0 <Xs:[] Zs:[0]>, -0.2 <Xs:[0] Zs:[1]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>},
 1: {0.8 <Xs:[] Zs:[1, 2]>,
  -0.2 <Xs:[0] Zs:[1]>,
  -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>},
 2: {0.8 <Xs:[] Zs:[1, 2]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>}}

In [36]:
H.backward(Rs)

In [62]:
H.descending()

[0.8 <Xs:[] Zs:[1, 2]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>, -0.2 <Xs:[0] Zs:[1]>]

In [38]:
H.imap

{0: {0.2 <Xs:[] Zs:[0]>, 1.0 <Xs:[0] Zs:[1]>},
 1: {0.8 <Xs:[] Zs:[1, 2]>, 1.0 <Xs:[0] Zs:[1]>, 0.5 <Xs:[1, 2] Zs:[]>},
 2: {0.8 <Xs:[] Zs:[1, 2]>, 0.5 <Xs:[1, 2] Zs:[]>}}

## SBRG

In [2]:
random.seed(2)
system = SBRG(TFIsing(16,J=1.,K=1.,h=1.,alpha=1.))
system.flow()

In [13]:
system.bit

5

In [42]:
system.phybits

[1, 3, 11, 12, 14]

In [29]:
system.H.terms

[0.2166551008243332 <Xs:[] Zs:[0, 1]>,
 0.027161847245396095 <Xs:[] Zs:[0, 1, 12, 14, 15]>,
 0.015037754441328979 <Xs:[] Zs:[0, 1, 13, 15]>,
 0.05913937003410543 <Xs:[] Zs:[0, 1, 15]>,
 0.005093107695763972 <Xs:[] Zs:[0, 12, 14]>,
 0.018658429413410674 <Xs:[] Zs:[0, 12, 14, 15]>,
 0.2274901235770828 <Xs:[] Zs:[1]>,
 0.5007067613106101 <Xs:[] Zs:[1, 2, 3]>,
 0.07863267363247047 <Xs:[] Zs:[2, 3]>,
 0.4009793653291901 <Xs:[] Zs:[3]>,
 0.480551921518253 <Xs:[] Zs:[3, 4, 5, 8]>,
 0.1346926943699165 <Xs:[] Zs:[7, 10, 11]>,
 0.12010267713206246 <Xs:[] Zs:[9, 10, 11]>,
 0.1621142087464852 <Xs:[] Zs:[10, 11]>,
 0.1113221322467988 <Xs:[] Zs:[11]>,
 0.5098359154470887 <Xs:[] Zs:[12]>,
 0.1216513704363341 <Xs:[] Zs:[12, 14]>,
 0.003699186652613278 <Xs:[1, 3, 11, 14] Zs:[4, 5, 6, 7, 8, 9, 12, 13]>,
 0.032574270684916855 <Xs:[1, 14] Zs:[0, 15]>,
 0.47459746510478773 <Xs:[3] Zs:[2]>,
 0.3097795710861902 <Xs:[11] Zs:[10]>]

In [30]:
system.H.imap[12]

{0.027161847245396095 <Xs:[] Zs:[0, 1, 12, 14, 15]>,
 0.005093107695763972 <Xs:[] Zs:[0, 12, 14]>,
 0.018658429413410674 <Xs:[] Zs:[0, 12, 14, 15]>,
 0.5098359154470887 <Xs:[] Zs:[12]>,
 0.1216513704363341 <Xs:[] Zs:[12, 14]>,
 0.003699186652613278 <Xs:[1, 3, 11, 14] Zs:[4, 5, 6, 7, 8, 9, 12, 13]>}

In [31]:
sorted(pert)

[0.007045123139215124 <Xs:[] Zs:[0, 1, 12, 13, 15]>,
 0.00856463245759557 <Xs:[] Zs:[0, 1, 13, 14, 15]>,
 0.01940534643017358 <Xs:[] Zs:[0, 1, 14, 15]>,
 0.004839543187700993 <Xs:[] Zs:[0, 12, 13, 15]>,
 0.005883347650036554 <Xs:[] Zs:[0, 13, 14, 15]>,
 0.013330215847949946 <Xs:[] Zs:[0, 14, 15]>,
 0.061253569539769494 <Xs:[] Zs:[11, 12]>,
 0.027034524310690415 <Xs:[] Zs:[11, 12, 13]>,
 0.02223814667143217 <Xs:[] Zs:[11, 13, 14]>,
 0.014967263984271503 <Xs:[] Zs:[11, 14]>,
 0.2526176011839408 <Xs:[] Zs:[12]>,
 0.03155340934786816 <Xs:[] Zs:[12, 13]>,
 0.15698400956292058 <Xs:[] Zs:[12, 13]>,
 0.0383589255302439 <Xs:[] Zs:[13, 14]>,
 0.12913241563304514 <Xs:[] Zs:[13, 14]>,
 0.056993142701065765 <Xs:[] Zs:[14]>,
 0.08691187185078415 <Xs:[] Zs:[14]>]

In [32]:
system.H.extend(pert)

In [33]:
system.H.terms

[0.2166551008243332 <Xs:[] Zs:[0, 1]>,
 0.007045123139215124 <Xs:[] Zs:[0, 1, 12, 13, 15]>,
 0.027161847245396095 <Xs:[] Zs:[0, 1, 12, 14, 15]>,
 0.00856463245759557 <Xs:[] Zs:[0, 1, 13, 14, 15]>,
 0.015037754441328979 <Xs:[] Zs:[0, 1, 13, 15]>,
 0.01940534643017358 <Xs:[] Zs:[0, 1, 14, 15]>,
 0.05913937003410543 <Xs:[] Zs:[0, 1, 15]>,
 0.004839543187700993 <Xs:[] Zs:[0, 12, 13, 15]>,
 0.005093107695763972 <Xs:[] Zs:[0, 12, 14]>,
 0.018658429413410674 <Xs:[] Zs:[0, 12, 14, 15]>,
 0.005883347650036554 <Xs:[] Zs:[0, 13, 14, 15]>,
 0.013330215847949946 <Xs:[] Zs:[0, 14, 15]>,
 0.2274901235770828 <Xs:[] Zs:[1]>,
 0.5007067613106101 <Xs:[] Zs:[1, 2, 3]>,
 0.07863267363247047 <Xs:[] Zs:[2, 3]>,
 0.4009793653291901 <Xs:[] Zs:[3]>,
 0.480551921518253 <Xs:[] Zs:[3, 4, 5, 8]>,
 0.1346926943699165 <Xs:[] Zs:[7, 10, 11]>,
 0.12010267713206246 <Xs:[] Zs:[9, 10, 11]>,
 0.1621142087464852 <Xs:[] Zs:[10, 11]>,
 0.1113221322467988 <Xs:[] Zs:[11]>,
 0.061253569539769494 <Xs:[] Zs:[11, 12]>,
 0.027034524

In [47]:
system.H.imap[12]

{0.007045123139215124 <Xs:[] Zs:[0, 1, 12, 13, 15]>,
 0.027161847245396095 <Xs:[] Zs:[0, 1, 12, 14, 15]>,
 0.004839543187700993 <Xs:[] Zs:[0, 12, 13, 15]>,
 0.005093107695763972 <Xs:[] Zs:[0, 12, 14]>,
 0.018658429413410674 <Xs:[] Zs:[0, 12, 14, 15]>,
 0.061253569539769494 <Xs:[] Zs:[11, 12]>,
 0.027034524310690415 <Xs:[] Zs:[11, 12, 13]>,
 0.7624535166310296 <Xs:[] Zs:[12]>,
 0.18853741891078873 <Xs:[] Zs:[12, 13]>,
 0.1216513704363341 <Xs:[] Zs:[12, 14]>,
 0.003699186652613278 <Xs:[1, 3, 11, 14] Zs:[4, 5, 6, 7, 8, 9, 12, 13]>}

In [6]:
random.seed(2)
system = SBRG(TFIsing(128,J=1.,K=1.,h=1.,alpha=1.))
% snakeviz system.flow()

 
*** Profile stats marshalled to file '/var/folders/tl/lwpcq5qj049ftcj7pvhkzv_h0000gn/T/tmp8m4al3'. 


In [134]:
Term(Pauli()) != Term(Pauli(),2.)

False

In [48]:
3//2

1

In [139]:
f(1)

[1]


In [5]:
H = Poly([Term(Pauli([1,3,0]),1.0),
      Term(Pauli([0,1,1]),0.5),
      Term(Pauli([3,0,0]),0.2),
      Term(Pauli([0,3,3]),0.8)])

In [13]:
A = H.terms[0]

In [21]:
cmp(1,0)

NameError: name 'cmp' is not defined

In [29]:
set(H.terms)

{0.8 <Xs:[] Zs:[1, 2]>,
 1.0 <Xs:[0] Zs:[1]>,
 0.2 <Xs:[] Zs:[0]>,
 0.5 <Xs:[1, 2] Zs:[]>}

In [30]:
H.terms

[0.8 <Xs:[] Zs:[1, 2]>,
 1.0 <Xs:[0] Zs:[1]>,
 0.2 <Xs:[] Zs:[0]>,
 0.5 <Xs:[1, 2] Zs:[]>]